<a href="https://colab.research.google.com/github/cheffjiu/pytorch-tutorials-zh/blob/main/quickstart_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/intro.ipynb) \|\| **Quickstart** \|\|
[Tensors](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/tensorqs_tutorial.ipynb) \|\| [Datasets &
DataLoaders](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/data_tutorial.ipynb) \|\|
[Transforms](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/transforms_tutorial.ipynb) \|\| [Build
Model](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/buildmodel_tutorial.ipynb) \|\|
[Autograd](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/autogradqs_tutorial.ipynb) \|\|
[Optimization](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/optimization_tutorial.ipynb) \|\| [Save & Load
Model](https://github.com/cheffjiu/pytorch-tutorials-zh/blob/main/saveloadrun_tutorial.ipynb)

Quickstart
==========

本节介绍了机器学习中常见任务的API。
请参考每个部分中的链接以深入了解。

Working with data（处理数据）
-----------------

PyTorch有两个[用于处理数据的基本类](https://pytorch.org/docs/stable/data.html)：`torch.utils.data.DataLoader`和`torch.utils.data.Dataset`。`Dataset`存储样本及其相应的标签，而`DataLoader`则把`Dataset`包装了一个可迭代对象。

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch提供特定领域的库，例如
[TorchText](https://pytorch.org/text/stable/index.html),
[TorchVision](https://pytorch.org/vision/stable/index.html), and
[TorchAudio](https://pytorch.org/audio/stable/index.html), 所有这些都包含数据集。在本教程中，我们将使用一个TorchVision数据集。

`torchvision.datasets`模块包含许多现实世界视觉数据的`Dataset`对象，如CIFAR、COCO（[完整列表在此](https://pytorch.org/vision/stable/datasets.html)）。在本教程中，我们使用FashionMNIST数据集。每个TorchVision `Dataset`都包含两个参数：`transform`和`target_transform`，分别用于修改样本和标签。   

代码如下：

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),#把样本数据变换成pytorch的张量
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将`Dataset`作为参数传递给`DataLoader`。`DataLoader`会把`Dataset`包装一个可迭代的数据集，并支持自动批处理、采样、打乱和多进程数据加载。在这里，我们定义批大小为64，即数据加载器可迭代对象中的每个元素将返回包含64个特征和标签的一批数据。      
代码如下：


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#X获取测试数据的特征，y获取标签。
for X, y in test_dataloader:
    #Fasion-minist数据集的图片都是单通道，高28，宽28，
    #N为test_dataloader采样的图片批量-——batch-size
    print(f"Shape of X [N, C, H, W]: {X.shape}")

    #每个图片有一个label，batch-size为64
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

从 [loading data in PyTorch](data_tutorial.html)阅读更多相关内容。


------------------------------------------------------------------------


Creating Models（构建模型）
===============

在PyTorch中定义神经网络模型时，我们需要创建一个继承自[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)的类。
并且在 `__init__` 函数中定义模型的各层，并在 `forward` 函数中指定数据将如何在模型中传递。为了加速神经网络中的运算，我们将其移动到[加速器](https://pytorch.org/docs/stable/torch.html#accelerators)，如CUDA、MPS、MTIA或XPU上。如果当前的加速器可用，我们就会使用它。否则，我们就使用CPU。    
代码如下：

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()#X [N, C, H, W]->[N,C*H*W]
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1*28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10) #把每张图片从512维空间映射到10维空间，Fusion-minist的类别为10类
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device) #实例化一个模型，并把模型移动到加速设备上
print(model)

从 [building neural networks in
PyTorch](buildmodel_tutorial.html)阅读更多信息


------------------------------------------------------------------------


Optimizing the Model Parameters（优化模型）
===============================
为了训练模型，我们需要 [loss
function](https://pytorch.org/docs/stable/nn.html#loss-functions) and an
[optimizer](https://pytorch.org/docs/stable/optim.html).


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型对训练数据集（以批次形式输入）进行预测，并反向传播预测误差以调整模型的参数。


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还会根据测试数据集来检查模型的性能，以确保它在学习。

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程会进行多次迭代（*epoch*）。在每一轮次中，模型学习参数以做出更好的预测。我们会打印每一轮次中模型的准确率和损失值；我们希望看到每一轮次中准确率上升，损失值下降。

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

从 [Training your model](optimization_tutorial.html)阅读更多内容


------------------------------------------------------------------------


Saving Models（保存模型）
=============

保存模型的一种常见方法是序列化内部状态字典（其中包含模型参数）。


In [ ]:
torch.save(model.state_dict(), "model.pth")#model.pth为存储模型参数的位置
print("Saved PyTorch Model State to model.pth")

Loading Models（加载模型）
==============

加载模型的过程包括重新创建模型结构，并将状态字典加载到其中。

In [ ]:
model = NeuralNetwork().to(device)#实例化模型
model.load_state_dict(torch.load("model.pth", weights_only=True))#从model.pth加载训练好的模型参数

这个模型现在可用于进行预测。


In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]#x [N, C, H, W],虽然N=1，但是也是要考虑这个维度，故pred[0]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

从[Saving & Loading your
model](saveloadrun_tutorial.html)阅读更多相关内容
